### 1111人力銀行

In [ ]:
#抓個縣市每個工作連結
import requests
from bs4 import BeautifulSoup as bs
import time
cou_url='http://www.1111.com.tw/job-bank/category.asp?cat=1&d=140202&city=1'
cou_res = requests.get(cou_url)
cou_soup = bs(cou_res.text)
cou_divs = cou_soup.select('.step2')[0]
all_job= ""
page_number=0
for p in xrange(0,15):
    divs1 = cou_divs.select('a')[p]['href']
    url1111 = 'http://www.1111.com.tw'
    url2222 = url1111+divs1
    job_url = url2222+'&page=1'
    job_res = requests.get(job_url)
    job_soup = bs(job_res.text)  
    
    page_div = job_soup.select('#showTotalPageCssBottom')[0]
    page_div1 = page_div.text.split('/'+' ')[1].split(' ')[0]
    page_number = int(page_div1)
    if (page_number)>1: 
        for i in xrange(0,page_number):
            job_url1 = url2222+'&page={}'.format(i+1)
            job_res1 = requests.get(job_url1)
            job_soup1 = bs(job_res1.text)
            len_div = len(job_soup1.select('.positionCssA'))
            for pages in xrange(0,len_div):
                job_href1 = job_soup1.select('.positionCssA')[pages]['href']
                all_job += job_href1+'\n'
    else:
        job_href = job_soup.select('.positionCssA')[0]['href'] 
        all_job += job_href+'\n'
    f = open('E:\job_url\job.txt','w')
    f.write(all_job.encode('utf8'))
    f.close()
    time.sleep(0.5)

In [ ]:
#進入每個網頁後爬取裡面內容
import requests
from bs4 import BeautifulSoup as bs
import time
with open('E:\job_url\job.txt', 'r') as f:
    for url in f.readlines():
        url_content = url
        res_content = requests.get(url_content)
        soup_content = bs(res_content.text)
        divs_content = soup_content.select('#midblock dt')
        divs1_content = soup_content.select('#midblock dd')
        title = soup_content.select('#commonTop h1')[0] 
        paper = title.text+' '
        len_div = len(divs_content)
        for n in range(0,len_div):
            x = divs_content[n].text
            y = divs1_content[n].text    
            dic = {x:y}
            for ele in dic:
                paper += ele+dic[ele]
            name = url_content.split('No=')[1].split('&')[0]         
            f = open('E:\job\{}.txt'.format(name),'w')
            f.write(paper.encode('utf8'))
            f.close()
            time.sleep(0.1)

### 104人力銀行

In [ ]:
#查尋軟體相關 一共有15xxx筆 
# 問題:單次查詢最多找到3000筆，可能爬取時須細分更多類別爬取

#將網址存到文件裡
import os
import time
import requests
from bs4 import BeautifulSoup as bs

with open('104.txt', 'w') as f:
    index = 150
    for page in xrange(1, 150):
        res = requests.get('http://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&jobcat=2007001000&order=2&asc=0&page={}&psl=N_B'.format(page))
        response = res.text.encode('utf8') 
        soup = bs(response)
        domain = 'http://www.104.com.tw'
        print_area = soup.select('.job_name')
        # print print_area
        for tr in print_area:
            if tr.select('a')[0]['href'][0:4] != "http":
                f.write(domain + tr.select('a')[0]['href']  + '\n')
            else:
                continue
        time.sleep(1) 

In [ ]:
#從文件中抓出連結，爬取網頁資料

import re
import time
dic = {}
with open('104.txt', 'r') as f:
    for line in f.readlines():
        url = line.strip()
        m = re.match('.*jobno=(\w+)', url)
        try:  #抓出失效連結
            jobno = m.group(1)
        except:
            print url
            continue
        with open('gov/{}.txt'.format(jobno), 'w') as f:
            res = requests.get(url)
            soup = bs(res.text)
            time.sleep(1)
            try:
                f.write(soup.select('#job , #job h1')[0].prettify('utf-8'))
            except:
                print url

# cheers

### 方法一:進到網頁判斷有沒有下一頁，如果有則進行下一頁，如果沒有則抓當頁

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
url="http://www.cheers.com.tw/top"
res = requests.get(url)#連到TOP20網頁
soup = bs(res.text)#整理內容
divs = soup.select('.top20group')[0]#抓到TOP20標籤
y = len(divs.select('a'))#以a標籤算長度
x_url=""
for i in range(0,y):#寫個迴圈把20篇放到x_url
    all_page=""#all_page做完回圈後會變空字串
    x_url = divs.select('a')[i]['href']#連到每個文章內
    res1 = requests.get(x_url)#跟每個頁面取得連結
    soup1 = bs(res1.text)#整理內容    
    rec_number = soup1.select('.pages')#抓頁數pages標籤
    pages_number = rec_number[0]#把陣列裡的第一個東西放到變數裡
    i_len = pages_number.select("i")#用i算長度
    pages = len(pages_number.select('li'))#用li算長度
    pages = pages-2#算頁數(減掉下一頁跟不分頁閱讀)
    if (len(i_len)>0):#判斷頁數是否超過1
        for i in range(0,pages):#抓取1-20頁
            next_page = x_url.split('&')[0]+'&page={}'.format(i+1)#截取&前面字串加上字串&page={}用format方法網頁頁數+1
            #url="http://www.cheers.com.tw/article/article.action?id=555555&page=i"
            res2 = requests.get(next_page)
            soup2 = bs(res2.text)
            divs2 = soup2.select('.chw_article_main_content p')
            for div3 in divs2[1:(len(divs1)-1)]:#頭尾標題不要
                all_page += div3.text #p標籤字串相加存到all_page
    else:#抓取當頁
        res3 = requests.get(x_url)
        soup3 = bs(res3.text)
        divs3 = soup3.select('.chw_article_main_content p')
        for div4 in divs3[1:(len(divs1)-1)]:
            all_page += div4.text #p標籤字串相加
    name = x_url.split('id=')[1].split('&')[0]        
    f = open('E:\cheers\{}.txt'.format(name),'w')
    f.write(all_page.encode('utf-8'))
    f.close()
    time.sleep(3)

### 方法二:進到網頁後直接抓取當頁，在判斷有沒有下頁，如果有則進行下一頁

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
url="http://www.cheers.com.tw/top"
res = requests.get(url)#連到TOP20網頁
soup = bs(res.text)#整理內容
divs = soup.select('.top20group')[0]#抓到TOP20標籤
y = len(divs.select('a'))#以a標籤算長度
x_url=""
for i in range(0,y):#寫個迴圈把20篇放到x_url
    all_page=""
    x_url = divs.select('a')[i]['href']#連到每個文章內
    res1 = requests.get(x_url)#跟每個頁面取得連結
    soup1 = bs(res1.text)#整理內容    
    divs1 = soup1.select('.chw_article_main_content p')
    for div in divs1[1:(len(divs1)-1)]:
            all_page += div.text #p標籤字串相加
    if (len(i_len)>0):
        for i in range(0,pages):
            next_page = x_url.split('&')[0]+'&page={}'.format(i+1)#截取&前面字串加上字串&page={}用format方法網頁頁數+1
            #url="http://www.cheers.com.tw/article/article.action?id=555555&page=i"
            res2 = requests.get(next_page)
            soup2 = bs(res2.text)
            divs2 = soup2.select('.chw_article_main_content p')
            for div3 in divs2[1:(len(divs1)-1)]:
                all_page += div3.text #p標籤字串相加存到all_page
    else:
        continue
    name = x_url.split('id=')[1].split('&')[0]        
    f = open('E:\cheers\{}.txt'.format(name),'w')
    f.write(all_page.encode('utf-8'))
    f.close()
    time.sleep(3)